Импорт библиотек.

In [ ]:
import pandas as pd
import numpy as np
import gdown
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.figure import Figure
import gradio as gr

In [2]:
%pip install ultralytics -q

In [3]:
from ultralytics import YOLO

Загружаем лучшую модель (файл best.pt нужно загрузить в сессионное хранилище в корневой каталог /content, по ссылке из интернета модель не разворачивается).

In [25]:
# загружаем веса лучшей модели
model = YOLO('/content/best.pt')

Функция draw() для отрисовки результата предсказания:

*   запускает model для детекции объектов на изображении,
*   рисует bounding boxes (рамки вокруг обнаруженных объектов),
*   возвращает обработанное изображение и список строк с обнаруженными объектами : классы (двери/ окна) и их координаты.



In [35]:
#Функция для отрисовки результата предсказания

def draw(image_for_drawing):
  #detections = []
  results = model.predict(image_for_drawing, conf=0.25)
#Перебираем результаты
  for result in results:
    # Получаем bounding boxes в формате [x1, y1, x2, y2, conf, class_id]
       boxes = result.boxes.xyxy.cpu().numpy()  # Координаты в пикселях
       confidences = result.boxes.conf.cpu().numpy()  # Уверенность
       class_ids = result.boxes.cls.cpu().numpy().astype(int)  # ID классов

       door_boxes = boxes[class_ids == 1]  # Массив координат дверей
       windows_boxes = boxes[class_ids == 4]
       door_confs = confidences[class_ids == 1]  # Уверенность для дверей
       windows_confs = confidences[class_ids == 4]  # Уверенность для окон
       detections = []
 # Выводим координаты и добавляем надписи
       for box, conf in zip(door_boxes, door_confs):
          x1, y1, x2, y2 = box
          detections.append(f"Дверь: x1={x1:.0f}, y1={y1:.0f}, x2={x2:.0f}, y2={y2:.0f}, confidence={conf:.2f}")
          cv2.rectangle(image_for_drawing, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        # Добавляем текст
          text = f"Door: {conf:.2f}"
          cv2.putText(image_for_drawing, text, (int(x1), int(y1) - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

       for box, conf in zip(windows_boxes, windows_confs):
          x1, y1, x2, y2 = box
          detections.append(f"Окно: x1={x1:.0f}, y1={y1:.0f}, x2={x2:.0f}, y2={y2:.0f}, confidence={conf:.2f}")
          cv2.rectangle(image_for_drawing, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        # Добавляем текст
          text = f"window: {conf:.2f}"
          cv2.putText(image_for_drawing, text, (int(x1), int(y1) - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
  #return image_for_drawing
  return image_for_drawing, "\n".join(detections)


Создаем веб-интерфейс для интерактивной демонстрации модели детекции объектов ( дверей и окон) с помощью библиотеки Gradio. На вход будет подаваться изображение  (в формате NumPy-массива), при отправке изображения в интерфейсе будет вызываться функция draw(), которая выведет изображение с нарисованными bounding boxes, а в текстовом окне - классы объектов и координаты.

In [37]:
demo = gr.Interface(
    fn=draw,
    inputs=[gr.Image(type="numpy")],
    outputs=[gr.Image(label="Результат детекции"), gr.Textbox(label="Обнаруженные объекты", lines=20)],
    title="Детектор дверей и окон"
)

demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d4057dbdf7970ad157.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x640 2 bathrooms, 7 doors, 1 perimeter, 1 stairs, 2 windows, 293.5ms
Speed: 5.7ms preprocess, 293.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d4057dbdf7970ad157.gradio.live
